In [ ]:
#https://medium.com/@yanis.labrak/how-to-train-a-custom-vision-transformer-vit-image-classifier-to-help-endoscopists-in-under-5-min-2e7e4110a353

In [1]:
!pip install hugsvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 708 kB 9.4 MB/s 
     |████████████████████████████████| 529 kB 54.1 MB/s 
     |████████████████████████████████| 548 kB 68.1 MB/s 
     |████████████████████████████████| 4.9 MB 36.4 MB/s 
     |████████████████████████████████| 5.9 MB 45.8 MB/s 
     |████████████████████████████████| 163 kB 68.2 MB/s 
     |████████████████████████████████| 6.6 MB 41.3 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220929150707 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.1 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   DOP  'How to get started with Drive.pdf'   obj_classes.zip


In [3]:
# this is where my zip is stored (I created a yolov3 folder where I will get my required files from)
!ls /mydrive

'Colab Notebooks'   DOP  'How to get started with Drive.pdf'   obj_classes.zip


In [4]:
# copy the .zip file into the root directory of cloud VM
!cp /mydrive/obj_classes.zip ../

In [5]:
# unzip the zip file and its contents should now be in /darknet/data/obj
!unzip ../obj_classes.zip -d data/

Archive:  ../obj_classes.zip
   creating: data/obj_classes/closed_manhole/
  inflating: data/obj_classes/closed_manhole/0001.jpg  
  inflating: data/obj_classes/closed_manhole/001.jpg  
  inflating: data/obj_classes/closed_manhole/002.jpg  
  inflating: data/obj_classes/closed_manhole/01.jpg  
  inflating: data/obj_classes/closed_manhole/01830928301.jpg  
  inflating: data/obj_classes/closed_manhole/09u3e091.jpg  
  inflating: data/obj_classes/closed_manhole/1.jpg  
  inflating: data/obj_classes/closed_manhole/100.jpg  
  inflating: data/obj_classes/closed_manhole/102.jpg  
  inflating: data/obj_classes/closed_manhole/107.jpg  
  inflating: data/obj_classes/closed_manhole/108.jpg  
  inflating: data/obj_classes/closed_manhole/11.jpg  
  inflating: data/obj_classes/closed_manhole/112.jpg  
  inflating: data/obj_classes/closed_manhole/117.jpg  
  inflating: data/obj_classes/closed_manhole/119.jpg  
  inflating: data/obj_classes/closed_manhole/12.jpg  
  inflating: data/obj_classes/closed

In [ ]:
import os
import shutil
import pandas as pd
metadata = pd.read_csv("HAM10000_metadata.csv").set_index('image_id').T.to_dict('list')
for current_dir in ["HAM10000_images_part_1","HAM10000_images_part_2"]:
    for image_path in os.listdir(current_dir):
        label = metadata[image_path.split(".")[0]][1]
        os.makedirs("./HAM10000/" + label, exist_ok=True)
        shutil.copy2(current_dir + "/" + image_path, "./HAM10000/" + label + "/" + image_path)

In [7]:
from hugsvision.dataio.VisionDataset import VisionDataset

train, test, id2label, label2id = VisionDataset.fromImageFolder(
  "./data/obj_classes",
  test_ratio   = 0.10,
  balanced     = False,
  augmentation = True,
)
print(id2label)
print(label2id)

Split Datasets...
train_ds:  928
+---------+----------------+---------------------------+--------------+-------+
| Dataset | closed_manhole | improperly_closed_manhole | open_manhole | Total |
+---------+----------------+---------------------------+--------------+-------+
|  Train  |      658       |            182            |      88      |  928  |
|  Test   |       81       |            12             |      11      |  104  |
+---------+----------------+---------------------------+--------------+-------+
{'0': 'closed_manhole', '1': 'improperly_closed_manhole', '2': 'open_manhole'}
{'closed_manhole': '0', 'improperly_closed_manhole': '1', 'open_manhole': '2'}


In [8]:
huggingface_model = 'google/vit-base-patch16-224-in21k'

In [11]:
from hugsvision.nnet.VisionClassifierTrainer import VisionClassifierTrainer
from transformers import ViTFeatureExtractor, ViTForImageClassification

trainer = VisionClassifierTrainer(
		model_name   = "Manhole_Detection",
		train        = train,
		test         = test,
		output_dir   = "./out/",
		max_epochs   = 30,
		batch_size   = 20, # On RTX 2080 Ti
		lr	     = 2e-5,
		fp16	     = True,
		model = ViTForImageClassification.from_pretrained(
				huggingface_model,
				num_labels = 3,
				label2id   = label2id,
				id2label   = id2label
				),
		feature_extractor = ViTFeatureExtractor.from_pretrained(
				huggingface_model,
				),
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--vit-base-patch16-224-in21k/snapshots/1ba429d32753f33a0660b80ac6f43a3c80c18938/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224-in21k",
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "closed_manhole",
    "1": "improperly_closed_manhole",
    "2": "open_manhole"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "closed_manhole": "0",
    "improperly_closed_manhole": "1",
    "open_manhole": "2"
  },
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.22.2"
}

loading weights file pytorch_model.bin from cac

{'0': 'closed_manhole', '1': 'improperly_closed_manhole', '2': 'open_manhole'}
{'closed_manhole': '0', 'improperly_closed_manhole': '1', 'open_manhole': '2'}
Trainer builded!
Start Training!


Epoch,Training Loss,Validation Loss
1,No log,0.578819
2,No log,0.458580
3,No log,0.366562
4,No log,0.312034
5,No log,0.328150
6,No log,0.309267
7,No log,0.311159
8,No log,0.308109
9,No log,0.333308
10,No log,0.338941


***** Running Evaluation *****
  Num examples = 104
  Batch size = 20
***** Running Evaluation *****
  Num examples = 104
  Batch size = 20
***** Running Evaluation *****
  Num examples = 104
  Batch size = 20
***** Running Evaluation *****
  Num examples = 104
  Batch size = 20
***** Running Evaluation *****
  Num examples = 104
  Batch size = 20
***** Running Evaluation *****
  Num examples = 104
  Batch size = 20
***** Running Evaluation *****
  Num examples = 104
  Batch size = 20
***** Running Evaluation *****
  Num examples = 104
  Batch size = 20
***** Running Evaluation *****
  Num examples = 104
  Batch size = 20
***** Running Evaluation *****
  Num examples = 104
  Batch size = 20
***** Running Evaluation *****
  Num examples = 104
  Batch size = 20
***** Running Evaluation *****
  Num examples = 104
  Batch size = 20
***** Running Evaluation *****
  Num examples = 104
  Batch size = 20
***** Running Evaluation *****
  Num examples = 104
  Batch size = 20
***** Running Evalua

Model saved at: ./out/MANHOLE_DETECTION/30_2022-10-09-07-22-54


In [12]:
ref, hyp = trainer.evaluate_f1_score()

100%|██████████| 104/104 [00:02<00:00, 36.68it/s]

                           precision    recall  f1-score   support

           closed_manhole     0.9518    0.9753    0.9634        81
improperly_closed_manhole     0.5833    0.5833    0.5833        12
             open_manhole     0.8889    0.7273    0.8000        11

                 accuracy                         0.9038       104
                macro avg     0.8080    0.7620    0.7822       104
             weighted avg     0.9026    0.9038    0.9023       104

Logs saved at: ./out/MANHOLE_DETECTION/30_2022-10-09-07-22-54


In [25]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(ref, hyp)
labels = list(label2id.keys())
df_cm = pd.DataFrame(cm, index = labels, columns = labels)

plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, annot_kws={"size": 8}, fmt="")
plt.savefig("./out/conf_matrix_1.jpg")

In [32]:
# Have to rename config.json to preprocessor_config.json. We also have to upload config.json (content of both are same) so that this works.
import os.path
from transformers import ViTFeatureExtractor, ViTForImageClassification
from hugsvision.inference.VisionClassifierInference import VisionClassifierInference

path = "./out/MANHOLE_DETECTION//30_2022-10-09-07-22-54/model/"
img  = "./data/obj_classes/open_manhole/000001.jpg"

classifier = VisionClassifierInference(
    feature_extractor = ViTFeatureExtractor.from_pretrained(path),
    model = ViTForImageClassification.from_pretrained(path),
)

label = classifier.predict(img_path=img)
print("Predicted class:", label)

loading configuration file ./out/MANHOLE_DETECTION//30_2022-10-09-07-22-54/model/preprocessor_config.json
Feature extractor ViTFeatureExtractor {
  "_name_or_path": "google/vit-base-patch16-224-in21k",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "do_normalize": true,
  "do_resize": true,
  "encoder_stride": 16,
  "feature_extractor_type": "ViTFeatureExtractor",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "closed_manhole",
    "1": "improperly_closed_manhole",
    "2": "open_manhole"
  },
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_size": 224,
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "closed_manhole": "0",
    "improperly_closed_manhole": "1",
    "open_manhole": "2"
  },
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hi

Model loaded!
Predicted class: open_manhole
